In [1]:

# Imports
import os
import json
import time
import pickle
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential, wait_random_exponential, retry_if_exception_type
from sklearn.metrics.pairwise import cosine_similarity

import google.generativeai as genai
from google.generativeai import GenerativeModel
from google.api_core import exceptions as google_exceptions
from google.api_core.exceptions import ResourceExhausted


import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

c:\Users\visha\miniconda3\envs\mchatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment
load_dotenv()
api_key = os.getenv("google_api_key") or "GOOGLE_API_KEY"

if api_key:
    print("API Key loaded successfully!")
else:
    print("Error: API Key not found.")

API Key loaded successfully!


In [3]:
# Configure Gemini
genai.configure(api_key=api_key)
model = GenerativeModel('gemini-1.5-flash')

# Use SentenceTransformer for embedding
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
# Load dataset
df = pd.read_csv("bitext/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")
if {'instruction', 'response'}.issubset(df.columns):
    df = df.rename(columns={"instruction": "prompt"})
df = df.dropna(subset=["prompt", "response"])
if "intent" in df.columns:
    df["intent"] = df["intent"].fillna("unknown")
df = df.sample(n=200, random_state=42)
print("\nFinal Dataset Info:")
df.info()


Final Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 9329 to 12181
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   flags     200 non-null    object
 1   prompt    200 non-null    object
 2   category  200 non-null    object
 3   intent    200 non-null    object
 4   response  200 non-null    object
dtypes: object(5)
memory usage: 9.4+ KB


In [5]:
import chromadb
from chromadb.config import Settings
import os

CHROMA_DB_DIR = "chroma_db"
collection_name = "bitext_support"

# Ensure the Chroma client is initialized with persistence
settings = Settings(chroma_db_impl="duckdb+parquet", persist_directory=CHROMA_DB_DIR)
chroma_client = chromadb.Client(settings)

# Create or load the collection
if collection_name in [col.name for col in chroma_client.list_collections()]:
    print(f"Loading existing Chroma collection: {collection_name}")
    collection = chroma_client.get_collection(name=collection_name)
else:
    print(f"Creating new Chroma collection: {collection_name}")
    collection = chroma_client.create_collection(name=collection_name)


ValueError: [91mYou are using a deprecated configuration of Chroma.

[94mIf you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/deployment/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/deployment/migration for more information or join our discord at https://discord.gg/MMeYNTmh3x for help![0m